In [1]:
import datetime as dt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import math
import netCDF4


In [2]:
## first, lets set some info about the times, level, etc., we want to plot

lev2plot = 500           #

year = 2021    
climoyr = 1980
month = 2
day = 1
total_days = 1           # how many days are we plotting?
first_fhr = 0            # setting to 0 = 00 UTC, 6 = 06 UTC, ...
hours = total_days * 24  # total hours in time range to plot
hr_inc = 24               # the delta time for the data

date1 = dt.datetime(year,month,day,0)  # first date to plot
datec = dt.datetime(climoyr,month,day,0)  # first date to plot

#create an array of times we want to plot...
times = [date1 + dt.timedelta(hours=x) for x in range(0,hours,hr_inc)]
ctimes = [datec + dt.timedelta(hours=x) for x in range(0,hours,hr_inc)]
#...and put them in the correct time units "hours since 00 UTC 1 Jan 1800 "
date_list = netCDF4.date2num(times,units="hours since 1900-01-01 00:00:00",calendar="gregorian") #change dates to netcdf times
Cdate_list = netCDF4.date2num(ctimes,units="hours since 1900-01-01 00:00:00",calendar="gregorian") #change dates to netcdf times


In [3]:
# Lets open our data file and get out data

# climo data is in /erai/climo/mean/VAR.nc and /erai/climo/stdev/VAR.nc


gfile = xr.open_dataset("/al11/andrea/teaching/atm525/era5_202102_g.nc")  ##Feb 2021 Z file
g_var = gfile["z"]
g_data  = (g_var.loc[dict(level=lev2plot,latitude=slice(90.,0) )]) / 9.81 ## div by gravity to get geopot. height

gcfile = xr.open_dataset("/langlab_rit/era5/z5dMean_00z.nc")  ##Climo mean of Z file
gc_var = gcfile["z"]
gc_data  = (gc_var.loc[dict(level=lev2plot,latitude=slice(90.,0) )]) / 9.81 ## div by gravity to get geopot. height

gsdfile = xr.open_dataset("/langlab_rit/era5/z5dStd_00z.nc")  ##Standard deviation of Z file
gsd_var = gsdfile["zstd"]
gsd_data  = (gsd_var.loc[dict(level=lev2plot,latitude=slice(90.,0) )]) / 9.81 ## div by gravity to get geopot. height

## convert 0 to 360 longitude to -180 to 180 to match the 2021 data for plotting/doing calculations
gc_data = gc_data.assign_coords({"longitude": (((gc_data.longitude + 180.) % 360.) - 180.)})  
gc_data = gc_data.sortby(gc_data.longitude)
gsd_data = gsd_data.assign_coords({"longitude": (((gsd_data.longitude + 180.) % 360.) - 180.)})
gsd_data = gsd_data.sortby(gsd_data.longitude)

lev = g_data["level"].values
lat = g_data["latitude"].values
lon = g_data["longitude"].values




In [4]:
for date in date_list: #loop through each day 
    ind = np.where(date_list == date)[0]
    for d in ind:
        height = g_data.loc[dict(time=times[d] )]
        climohght = gc_data.loc[dict(time=ctimes[d] )]
        stdhght = gsd_data.loc[dict(time=ctimes[d] )]
        print(height.coords)
        print(climohght.coords)
        anoms = (height - climohght) / stdhght  ## calculate the standardized geo. height anom.
                
        print("Date "+times[d].strftime("%H UTC %d %b %Y"))
        valid_label = (times[d].strftime("Valid: %H UTC %d %b %Y"))

        plt.Figure(figsize=(15,15),dpi=120)    ## <---Set fig size here!

        #Set plot as an orthographic projection looking down at the Earth from the pole with 0˚ at 6-oclock
        ax = plt.axes(projection=ccrs.Orthographic(0,90))     
        ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='dimgray',facecolor='none')
        ax.outline_patch.set_edgecolor('none')
        gl = ax.gridlines(color="grey",linestyle=":",linewidth=0.5)  
        glevs = range(1000,8000,100)  ## <---set contour interval for geo. heights
        

        [x,y] = np.meshgrid(lon,lat)  ## <--- set lat/lon as coords on grid
        
        plt.title(str(lev2plot)+"hPa Geopotential Height & Temp \n"+valid_label)
        hghtplot = ax.contour(x,y,height,glevs,transform=ccrs.PlateCarree(),extend='both',colors='black',linewidth=0.8)
        ax.clabel(hghtplot, inline=True, fmt="%.0f", fontsize=8)
        
        cmap = matplotlib.cm.bwr  ##<--- color map to use
        norm = matplotlib.colors.Normalize(vmin=-5, vmax=5)    ##<---min/max for standardized anom/ ±4 sigma might work
        anomlevs = np.arange(-4.,4.,.25)

        p2plot = ax.contourf(x,y,anoms,anomlevs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm, alpha=1)
        plt.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), label="Standardized Height Anomaly", alpha=1)

        plot_label = times[d].strftime("%Y%m%d%H")
        plt.savefig(f"{plot_label}_{lev2plot}hpaANOM.png",format='png')
        plt.clf()
        

Coordinates:
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
    level      int32 500
    time       datetime64[ns] 2021-02-01
Coordinates:
    time       datetime64[ns] 1980-02-01
    level      int32 500
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... 0.75 0.5 0.25 0.0
  * longitude  (longitude) float32 -180.0 -179.75 -179.5 ... 179.25 179.5 179.75
Date 00 UTC 01 Feb 2021


/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.
/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/cartopy/mpl/geoaxes.py:1478: UserWarning: The following kwargs were not used by contour: 'linewidth'
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)
